In [5]:
# ===== CSV 두 묶음 머지 후 저장 (Jupyter 셀 하나로) =====
import os, warnings
import pandas as pd

warnings.filterwarnings("ignore")

BASE = "./raw/drop"
OUT_DIR = os.path.join(BASE, "_merged")
os.makedirs(OUT_DIR, exist_ok=True)

# 파일 경로
fp_a1 = os.path.join(BASE, "파프리카_drop.csv")
fp_a2 = os.path.join(BASE, "파프리카_1_drop.csv")
fp_b1 = os.path.join(BASE, "청고추_drop.csv")
fp_b2 = os.path.join(BASE, "청고추_1_drop.csv")

# 인코딩 가변 로더
def read_csv_robust(path):
    for enc in ["utf-8-sig", "utf-8", "cp949", "euc-kr"]:
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except Exception:
            continue
    raise RuntimeError(f"CSV 로딩 실패: {path}")

# 열 스탠다드라이즈(양쪽에 없는 열은 NaN으로 채움)
def align_cols_union(df_list):
    all_cols = []
    for d in df_list:
        all_cols.extend(list(d.columns))
    all_cols = list(dict.fromkeys([str(c).strip() for c in all_cols]))  # 순서 유지 + 중복 제거
    out = [d.reindex(columns=all_cols) for d in df_list]
    return out

# 중복 제거 기준(있으면 사용)
DUP_KEYS = ["거래일자", "품목명", "세부 품목명", "세부 품목", "세부품목"]

def drop_dups(df):
    # 사용 가능한 키만 추림
    keys = [k for k in DUP_KEYS if k in df.columns]
    if len(keys) >= 2:  # 최소 2개 이상 있을 때 키로 중복 제거
        return df.drop_duplicates(subset=keys)
    return df.drop_duplicates()

def tidy_sort(df):
    # 날짜열 정렬 시도
    date_cols = [c for c in df.columns if ("거래일자" in str(c)) or (str(c).lower() in ["date", "날짜"])]
    for c in date_cols:
        try:
            df[c] = pd.to_datetime(df[c], errors="coerce")
        except Exception:
            pass
    # 대표 날짜열이 있으면 그걸로 정렬
    if date_cols:
        return df.sort_values(by=[date_cols[0], *(col for col in ["품목명","세부 품목명","세부 품목","세부품목"] if col in df.columns)])
    return df

def merge_and_save(paths, out_name):
    # 로드
    dfs = [read_csv_robust(p) for p in paths]
    # 열 맞춤
    dfs = align_cols_union(dfs)
    # 수직 결합
    merged = pd.concat(dfs, axis=0, ignore_index=True)
    # 공백/문자열 정리
    merged.columns = [str(c).strip() for c in merged.columns]
    # 중복 제거
    merged = drop_dups(merged)
    # 정렬
    merged = tidy_sort(merged)
    # 저장
    out_path = os.path.join(OUT_DIR, out_name)
    merged.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"[OK] {out_name} 저장: {len(merged):,}행  | 열 {len(merged.columns)}개")
    return merged

# === 실행 ===
df_pap = merge_and_save([fp_a1, fp_a2], "파프리카_merged.csv")
df_ch  = merge_and_save([fp_b1, fp_b2], "청고추_merged.csv")

# 화면 미리보기
display(df_pap.head(10))
display(df_ch.head(10))


[OK] 파프리카_merged.csv 저장: 7,308행  | 열 12개
[OK] 청고추_merged.csv 저장: 14,616행  | 열 12개


,거래일자,품목명,반입량,금액,전년 반입량,전년 반입량 증감률(%),전년 금액,전년 금액 증감률(%),평년 반입량,평년 반입량 증감률(%),평년 금액,평년 금액 증감률(%)
3653,2015-06-30,노랑파프리카,43270.0,86093000.0,63799.0,-32.2,90313600.0,-4.7,32579.0,32.8,76470468.0,12.6
7307,2015-06-30,빨강파프리카,65721.0,128669500.0,68325.0,-3.8,100405000.0,28.2,38961.0,68.7,93162765.0,38.1
3652,2015-07-01,노랑파프리카,65033.0,112367500.0,34855.0,86.6,54636500.0,105.7,31401.0,107.1,75523333.0,48.8
7306,2015-07-01,빨강파프리카,88696.0,140383200.0,34269.0,158.8,59262500.0,136.9,39165.0,126.5,90541181.0,55.0
3651,2015-07-02,노랑파프리카,62996.0,93907500.0,38815.0,62.3,75192000.0,24.9,31989.0,96.9,72338267.0,29.8
7305,2015-07-02,빨강파프리카,84710.0,116737000.0,41265.0,105.3,82105000.0,42.2,39641.0,113.7,89559813.0,30.3
3650,2015-07-03,노랑파프리카,41665.0,63535000.0,54185.0,-23.1,112994000.0,-43.8,33786.0,23.3,70849533.0,-10.3
7304,2015-07-03,빨강파프리카,65885.0,92834500.0,57130.0,15.3,112212500.0,-17.3,41022.0,60.6,87022230.0,6.7
3649,2015-07-04,노랑파프리카,53087.0,87212000.0,48499.0,9.5,101369500.0,-14.0,33106.0,60.4,67492550.0,29.2
7303,2015-07-04,빨강파프리카,75474.0,105660000.0,53554.0,40.9,121442000.0,-13.0,40763.0,85.2,81604697.0,29.5


,거래일자,품목명,반입량,금액,전년 반입량,전년 반입량 증감률(%),전년 금액,전년 금액 증감률(%),평년 반입량,평년 반입량 증감률(%),평년 금액,평년 금액 증감률(%)
3653,2015-06-30,녹광,16256.0,38140670.0,21615.0,-24.8,57638600.0,-33.8,13148.0,23.6,32533782.0,17.2
7307,2015-06-30,오이맛고추,42649.0,71647500.0,38964.0,9.5,76413450.0,-6.2,34241.0,24.6,58300685.0,22.9
10961,2015-06-30,청양,92727.0,193123810.0,91072.0,1.8,340061750.0,-43.2,70570.0,31.4,233850535.0,-17.4
14615,2015-06-30,청초(일반),6384.0,14622000.0,15099.0,-57.7,33851900.0,-56.8,NaN,0.0,NaN,0.0
3652,2015-07-01,녹광,12944.0,24033000.0,19510.0,-33.7,44965100.0,-46.6,13290.0,-2.6,33512835.0,-28.3
7306,2015-07-01,오이맛고추,42821.0,68398250.0,45659.0,-6.2,80354550.0,-14.9,34115.0,25.5,59756219.0,14.5
10960,2015-07-01,청양,86307.0,166001900.0,78212.0,10.4,259235640.0,-36.0,70938.0,21.7,232413767.0,-28.6
14614,2015-07-01,청초(일반),4523.0,10839600.0,7144.0,-36.7,12887300.0,-15.9,NaN,0.0,NaN,0.0
3651,2015-07-02,녹광,11733.0,23444750.0,10996.0,6.7,19306900.0,21.4,13150.0,-10.8,31424942.0,-25.4
7305,2015-07-02,오이맛고추,43057.0,68435700.0,36677.0,17.4,51068750.0,34.0,33284.0,29.4,59535195.0,14.9
